In [28]:
import pandas as pd
import os

In [29]:
df = pd.read_csv('./users.golden_label.csv', low_memory=False)
print df.shape
df.head()

(168457, 8)


,user_id,username,freq,male.prob,fname.gender,male.prob2,fname.gender2,pic.gender
0,5,billion,2.0,1.0000,male,0.920000,male,NaN
1,63,paddyshyam,NaN,NaN,NaN,NaN,NaN,NaN
2,64,ArmenSoft,1.0,1.0000,male,NaN,NaN,NaN
3,128,okbookman,1.0,0.9945,male,0.998834,male,NaN
4,215,revitupmedia,1.0,1.0000,male,NaN,NaN,male


In [3]:
df1 = df[((~df['fname.gender'].isnull())& abs(df['male.prob']-0.5)>0.4 )|\
   ((~df['fname.gender2'].isnull())& abs(df['male.prob2']-0.5)>0.4)|(~df['pic.gender'].isnull())]
df1.shape

(68786, 8)

In [9]:
df2 = df1[['fname.gender','fname.gender2','pic.gender']].fillna(0).applymap(lambda x: {'male':1,'female':-1,0:0}[x])
print df2.shape
df2.head()

(68786, 3)


,fname.gender,fname.gender2,pic.gender
0,1,1,0
2,1,0,0
3,1,1,0
4,1,0,1
5,0,0,1


### Pic prediction comes priority

In [10]:
df_pic = df2[df2['pic.gender']!=0]
df3 = df2.drop(df_pic.index)
print df_pic.shape,df3.shape

(12339, 3) (56447, 3)


### Firstname Pred labels that contradict with each other

In [11]:
conflict = df3[df3.sum(1)==0]
print conflict.shape
conflict.head()

(2193, 3)


,fname.gender,fname.gender2,pic.gender
46,1,-1,0
75,-1,1,0
103,-1,1,0
115,1,-1,0
182,-1,1,0


#### original conflict instances

In [12]:
df.loc[conflict.index,:].head()

,user_id,username,freq,male.prob,fname.gender,male.prob2,fname.gender2,pic.gender
46,2200,gaxco,2.0,0.9677,male,0.498580,female,NaN
75,4348,jmawebco,2.0,0.0000,female,0.522277,male,NaN
103,6614,Ray33,2.0,0.0000,female,0.700000,male,NaN
115,7998,ChristianSEO,5.0,0.8397,male,0.397135,female,NaN
182,13673,kolorvize,1.0,0.0000,female,0.522277,male,NaN


In [13]:
df4 = df3.drop(conflict.index)
df4.shape

(54254, 3)

In [14]:
df4['label']=df4.sum(1)>0
df4.head()

,fname.gender,fname.gender2,pic.gender,label
0,1,1,0,True
2,1,0,0,True
3,1,1,0,True
7,1,1,0,True
8,1,1,0,True


In [15]:
df_pic.loc[:,('label')]=df_pic['pic.gender']>0
df_pic.head()

/home/shj16110/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/shj16110/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,fname.gender,fname.gender2,pic.gender,label
4,1,0,1,True
5,0,0,1,True
6,0,0,1,True
9,0,0,1,True
14,0,0,1,True


#### final training dataset

In [16]:
df_train = pd.concat([df4, df_pic])
df_train.shape

(66593, 4)

In [17]:
df_Need_Pred = df.drop(df_train.index)
df_Need_Pred.shape

(101864, 8)

In [18]:
df_Need_Pred0 = df_Need_Pred[['user_id','username']]
df_Need_Pred0.to_csv('Need_Pred.csv', index_label='index')

In [19]:
df_train1 = df_train.reset_index()
df_train2 = df_train1.sort_values('index').reset_index(drop=True)
df_train2['label'] = df_train2['label'].map({True:1,False:0})
df_train3 = df_train2[['index','label']]
df_train3.head()

,index,label
0,0,1
1,2,1
2,3,1
3,4,1
4,5,1


In [25]:
df4 = df_train3.set_index('index').join(df, how='inner')[['label','username']]
df4.shape

(66593, 2)

In [26]:
df4.head()

,label,username
0,1,billion
2,1,ArmenSoft
3,1,okbookman
4,1,revitupmedia
5,1,AnsonSequeira


In [27]:
df4.to_csv('New_Training_set.csv', index=False)

In [33]:
df['username'].value_counts()

RSwapnil            2
devservice          2
YuanRemo            2
shane50             2
thecoldfusioner     2
vishalweb           2
qclservice          2
therealphoenix      2
bchandra1955        2
anum28k             2
rejohn2naz          2
PHPsoldier          2
mrcool18t           2
useradds            2
contentside         2
k00lcaptchrz        2
bogdanAR            2
seowork12           2
StuartSimon         2
BIGOT               2
uniform2233         2
balaprabu           2
wynnewebdesign      2
joxster             2
leadlancer          2
moshthepitt         2
rashinrohanifar     2
tjoozey             2
samuraig            2
VeeHaywood          2
                   ..
DerekT              1
pro3innovative      1
TrishLAustin        1
ahmedhayman         1
rgeorgiou           1
flasher3015         1
applemae            1
indhar15            1
usmanalik           1
cb990094            1
gearshop            1
javakolkata         1
SouthPawScripter    1
protein007          1
geraldhall